In [1]:
%pylab inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
# #obs
N = train.feat_1.count()
print N

61878


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.calibration import CalibratedClassifierCV

# === Parameter tuning - Using Hyperopt ===
from hyperopt import hp, fmin, tpe
from time import time

from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss

In [7]:
log_loss(["spam", "ham", "ham", "spam"],  [[.1, .9], [.9, .1], [.8, .2], [.35, .65]])

0.21616187468057912

[.1, .9] - 0.1 predicted probability for class 0 and 0.9 predicted probability for class 1 <br/>
class 0 - ham, class 1 - spam <br/>
By default, spam is 1, and ham is 0

In [19]:
y_true = [0, 0, 1, 1]
y_pred = [[.9, .1], [.8, .2], [.3, .7], [.01, .99]]
log_loss(y_true, y_pred)   

0.17380733669106746

In [65]:
log_loss(["Class_1", "Class_2"], [ [0.9, 0.1], [0.1, 0.9] ]  )

0.10536051565782628

In [66]:
-(1*log(0.9) + 1*log(0.9)) / 2.0

0.10536051565782628

#### Random Forest

In [31]:
space = (
    hp.choice( 'max_depth', np.arange(6, 42, step=2)),
    hp.choice( 'max_features', np.arange(0.1, 1.1, step=0.1) )
)

In [32]:
def run_wrapper( params ):
    global run_counter
    
    run_counter += 1
    print "run " + str(run_counter)
    
    s = time()
    train_error, test_error = run_test( params )
    
    print "Test error:", str(test_error) + ' | param: ' + str( params )
    print "Train error: %r " % train_error
    print "elapsed: {}s \n".format( int( round( time() - s )))
    
    return test_error

In [10]:
X_train = train.drop(["target", "id"], axis = 1)
y_train = train.target

In [11]:
y_train[:5]

0    Class_1
1    Class_1
2    Class_1
3    Class_1
4    Class_1
Name: target, dtype: object

In [17]:
import sklearn
sklearn.__version__

'0.16.1'

In [45]:
train1 = pd.read_csv("train1.csv")
test1  = pd.read_csv("test1.csv")

In [46]:
X_train1 = train1.drop(["target", "id"], axis = 1)
y_train1 = train1.target

In [48]:
X_train1[:5]

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,3,1,4,0,0,1,...,3,0,1,2,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [47]:
y_train1[:5]

0    Class_1
1    Class_1
2    Class_1
3    Class_1
4    Class_1
Name: target, dtype: object

In [11]:
X_train = train.drop(["target", "id"], axis = 1)
y_train = train.target

In [25]:
def run_calibrated_rf( X_train, y_train, folds = 5, ntree = 300, max_depth = 40, min_leaf = 4, max_features = 0.2, njobs = 16, calibrated_folds = 5 ):
    
    kf = StratifiedKFold(y_train.values, n_folds=folds, shuffle=False)
    X = X_train.values
    y = y_train.values
    
    train_cv_error = np.empty( folds ) 
    test_cv_error  = np.empty( folds )

    rf_model = RandomForestClassifier(n_estimators = ntree, 
                                      max_depth    = max_depth, 
                                      min_samples_leaf = min_leaf,
                                      max_features = max_features,
                                      n_jobs       = njobs)   # running 4 jobs on 4 cores

    for index, (train_index, test_index) in enumerate(kf):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        rf_fit = rf_model.fit(X_train, y_train)
        # use calibrated classifier on training part
        cal_rf_fit = CalibratedClassifierCV(rf_fit, method="isotonic", cv = calibrated_folds)
        cal_rf_fit.fit(X_train, y_train)  
        
        train_cv_error[index] = log_loss( y_train, cal_rf_fit.predict_proba(X_train) ) 
        test_cv_error[index]  = log_loss( y_test, cal_rf_fit.predict_proba(X_test) ) 

    train_error = np.mean(train_cv_error)
    test_error  = np.mean(test_cv_error)
    return train_error, test_error

In [36]:
run_counter = 0

best = fmin( run_wrapper,           # function to minimize
             space,                 # search space
             algo = tpe.suggest,    # algo
             max_evals = 50         # max iterations
             )

run 1
Test error: 0.730648194454 | param: (10, 0.30000000000000004)
Train error: 0.65476389828382797 
elapsed: 40s 

run 2
Test error: 0.578089280098 | param: (20, 0.20000000000000001)
Train error: 0.35544117697215843 
elapsed: 42s 

run 3
Test error: 0.690190668565 | param: (12, 0.30000000000000004)
Train error: 0.57755219659290469 
elapsed: 44s 

run 4
Test error: 0.573509847469 | param: (36, 0.70000000000000007)
Train error: 0.23713032314355292 
elapsed: 117s 

run 5
Test error: 0.896762270424 | param: (10, 1.0)
Train error: 0.8382386602706362 
elapsed: 95s 

run 6
Test error: 0.620273717762 | param: (16, 0.30000000000000004)
Train error: 0.45109663144334028 
elapsed: 50s 

run 7
Test error: 0.601767080585 | param: (22, 0.80000000000000004)
Train error: 0.338341934176006 
elapsed: 123s 

run 8
Test error: 0.611086284429 | param: (26, 1.0)
Train error: 0.29920999620043409 
elapsed: 160s 

run 9
Test error: 0.568294133603 | param: (30, 0.40000000000000002)
Train error: 0.2491051267742

KeyboardInterrupt: 

In [110]:
best

{'max_depth': 2, 'max_features': 1, 'min_samples_leaf': 2}

    run 6
    Test error: 1.06310537073 | param: (7, 7, 0.5)
    Train error: 1.0530566950749782 

In [22]:
run_test( (40, 0.2, 5, 10) )

# 40, 0.2, 3, 10 - with 5cv: (0.57271343015443876, 0.57665166874091223)
# 40, 0.2, 5 with 5cv, ntree = 200: (0.27080722352332665, 0.50759596717809896) --> correct
# min leaf: 3 - with 500 trees: (0.26883396108621033, 0.50694138369562702) - slightly reduced

(0.26883396108621033, 0.50694138369562702)

In [29]:
run_calibrated_rf(X_train, y_train, folds = 5, ntree = 200, max_depth = 40, min_leaf = 3, max_features = 0.2, 
                  njobs = 16, calibrated_folds = 3)

# folds   ntree   depth   leaf   max_features    calibrated_folds    train    test 
# 5       500     40      3      0.2             5                   .26883   .50694
# 5       500     40      3      0.2             3                   .26508   .51454 

# 5       200     50      4      0.2             5                   .27154   .51558
# 5       200     30      4      0.2             5                   .28921   .51735
# 5       200     20      3      0.2             5                   .3611    .54512


(0.26508044208850962, 0.51454057385879537)

In [38]:
rf_model = RandomForestClassifier(n_estimators=500, 
                                    max_depth=40, 
                                    min_samples_leaf=4,
                                    max_features=0.2,
                                    n_jobs = 16)   # running 4 jobs on 4 cores

rf_fit = rf_model.fit(X_train, y_train)

**RF** <br/>
Without calibration: 

    10, 4, 1.0: (0.91251429652300209, 0.94094987502326144)
    40, 4, 1.0: (0.32113909387768685, 0.60717827845190864)
    60, 4, 1.0: (0.31771690448338419, 0.6092925576496071)

With calibration:

    10, 4, 1.0: (0.8356862096817943, 0.83290941743252522)
    40, 4, 1.0: (0.25332064875228538, 0.60348477155300828)
    40, 4, 0.5: (0.23218622276408657, 0.56390228229675121)  # ntree = 300
    20, 4, 0.5: (0.36511831993499538, 0.59753675035788323)  # ntree = 300
    30, 4, 0.5: (0.24953190108702428, 0.56458202652808265)  # ntree = 500
    40, 4, 0.2: (0.23873372608827595, 0.548096151395)       # ntree = 500
    60, 4, 0.2: (0.3088395524161624, 0.56883823507334863)
    60, 4, 1.0: (0.30861626646231655, 0.56614864405879306)
    
With calibration, feature engineering:

    40, 4, 1.0: (0.31838763002858245, 0.57374920366401549)
    40, 4, 0.2: (0.31871012865244258, 0.57653341441142436)
    20, 4, 0.2: (0.3990457711953671, 0.60149029853276681)
    20, 4, 1.0: (0.39945648233825964, 0.60125135890332582)
    60, 4, 1.0: (0.31742162179208683, 0.57465458249032864)
    80, 4, 1.0: (0.31777607892982829, 0.57127352932674513)
    80, 4, 0.2: (0.31740112731579068, 0.56912122044373681)
   
With choice of calibration folds and CV folds

    40, 1.0, 3 (cali folds), 10 (CV folds): (0.25198958289714674, 0.59369842025685893) 
    40, 0.2, 3, 10: (0.23919816211764594, 0.53363464574189834)
    40, 0.2, 2, 10: (0.24310196201783557, 0.54914480378545016)
    (0.23618684491052028, 0.53709911088693973)


**Extra Forests** <br/>
    40, 4, 0.2: (0.4457263724445687, 0.61359812138729486)
    20, 4, 0.2: (0.61621853972602847, 0.70603863875428208)
    20, 4, 1.0: (0.61472915058327471, 0.70238058124675906)

In [111]:
rf_model = RandomForestClassifier(n_estimators=200, 
                                    max_depth=7, 
                                    min_samples_leaf=7,
                                    max_features=0.5)
rf_fit = rf_model.fit(X_train, y_train)

In [112]:
rf_fit

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=7, max_features=0.5,
            max_leaf_nodes=None, min_density=None, min_samples_leaf=7,
            min_samples_split=2, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)

In [115]:
X_test = test.drop(['id'], axis=1)
predicted_test = rf_fit.predict_proba(X_test)

In [116]:
predicted_test[:5]

array([[  2.70372669e-02,   2.84089152e-01,   1.50174340e-01,
          2.30835459e-01,   9.81831444e-03,   1.07760813e-01,
          4.96887087e-02,   7.14070242e-02,   6.91889225e-02],
       [  2.86511083e-02,   2.08242012e-01,   1.20962869e-01,
          1.06313083e-01,   1.01716053e-02,   2.29207343e-01,
          3.76515554e-02,   2.06666135e-01,   5.21342891e-02],
       [  7.84380067e-04,   6.18251546e-04,   5.71601726e-04,
          8.22772943e-05,   1.64245445e-04,   9.90329200e-01,
          1.38779204e-03,   5.10589303e-03,   9.56358857e-04],
       [  1.93503294e-02,   4.32448514e-01,   2.68008976e-01,
          1.00867602e-01,   5.71273233e-03,   3.37295935e-02,
          3.69000466e-02,   5.27885999e-02,   5.01936059e-02],
       [  6.59828963e-02,   1.44940851e-01,   9.25814024e-02,
          2.25283560e-02,   9.90778314e-03,   8.37239250e-02,
          6.36447568e-02,   3.35690661e-01,   1.80999368e-01]])

In [117]:
train.target.unique()

array(['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9'], dtype=object)

In [118]:
predicted_test = pd.DataFrame(columns=train.target.unique(), data=predicted_test)

In [119]:
predicted_test[:5]

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,0.027037,0.284089,0.150174,0.230835,0.009818,0.107761,0.049689,0.071407,0.069189
1,0.028651,0.208242,0.120963,0.106313,0.010172,0.229207,0.037652,0.206666,0.052134
2,0.000784,0.000618,0.000572,0.000082,0.000164,0.990329,0.001388,0.005106,0.000956
3,0.019350,0.432449,0.268009,0.100868,0.005713,0.033730,0.036900,0.052789,0.050194
4,0.065983,0.144941,0.092581,0.022528,0.009908,0.083724,0.063645,0.335691,0.180999


In [120]:
submit = test.filter(['id'], axis=1).join(predicted_test)

In [122]:
submit.to_csv("submissions/04May_RF_02.csv", index=False)

This model scores ￼1.05984 on LB, which is pretty consistent with 5 CV score. 

In [126]:
run_test( (20, 7, 1.0) )

(0.5351777727832947, 0.66301274716062719)

In [141]:
leaves = [5, 10, 20, 30, 45, 60, 80, 100, 120, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600]

In [142]:
space = (
    hp.choice( 'max_depth', np.arange(4, 93, step=2)),
    hp.choice( 'min_samples_leaf', leaves ),
    hp.choice( 'max_features', np.arange(0.1, 1.1, step=0.1) )
)

array([ 10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,  70,
        75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125, 130, 135,
       140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200,
       205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265,
       270, 275, 280, 285, 290, 295, 300, 305, 310, 315, 320, 325, 330,
       335, 340, 345, 350, 355, 360, 365, 370, 375, 380, 385, 390, 395,
       400, 405, 410, 415, 420, 425, 430, 435, 440, 445, 450, 455, 460,
       465, 470, 475, 480, 485, 490, 495, 500, 505, 510, 515, 520, 525,
       530, 535, 540, 545, 550, 555, 560, 565, 570, 575, 580, 585, 590, 595])